# Value at Risk (VaR)

In [3]:
import numpy as np
import yfinance as yf
import pandas as pd
import datetime
from scipy.stats import norm

In [4]:


def download_data(stock, start_date, end_date):
    data = {}

    # Download stock data using yfinance
    ticker = yf.download(stock, start_date, end_date)

    # Extract the 'Adj Close' prices and store them in the data dictionary
    data[stock] = ticker['Adj Close']

    # Create a DataFrame from the data dictionary
    return pd.DataFrame(data)


In [5]:
# c = confidence level

#This is how we calculate the value at risk tomorrow (n=1)
def calculate_VaR(position, c, mu, sigma):
    # Calculate the z-score corresponding to the confidence level (1 - c)
    v = norm.ppf(1 - c)

    # Calculate the VaR using the position, mean (mu), and standard deviation (sigma)
    var = position * (mu - sigma * v)

    return var

# This is how we calculate VaR for any number of days in the future
def calculate_VaR_n(position, c, mu, sigma, n):
    # Calculate the z-score corresponding to the confidence level (1 - c)
    v = norm.ppf(1 - c)

    # Calculate the VaR using the position, mean (mu), standard deviation (sigma), and number of days (n)
    var = position * (mu * n - sigma * np.sqrt(n) * v)

    return var


In [6]:
# Define the download_data function (as mentioned earlier)

start = datetime.datetime(2014, 1, 1)
end = datetime.datetime(2018, 1, 1)

# Download stock data for the symbol 'C' from start to end dates
stock_data = download_data('C', start, end)

# Calculate the logarithmic returns using the 'Adj Close' prices
stock_data['Returns'] = np.log(stock_data['C'] / stock_data['C'].shift(1))

# Remove the first row with NaN values
stock_data = stock_data[1:]

# Print the resulting stock data
print(stock_data)

[*********************100%***********************]  1 of 1 completed
                    C   Returns
Date                           
2014-01-03  43.411266  0.021388
2014-01-06  43.744560  0.007648
2014-01-07  44.045353  0.006853
2014-01-08  44.557514  0.011561
2014-01-09  44.874577  0.007091
...               ...       ...
2017-12-22  63.072460 -0.004362
2017-12-26  62.479244 -0.009450
2017-12-27  62.571152  0.001470
2017-12-28  62.729912  0.002534
2017-12-29  62.170120 -0.008964

[1006 rows x 2 columns]


In [7]:
# Define the calculate_VaR_n function (as mentioned earlier)

# This is the investment (stocks or whatever)
S = 1e6
# Confidence level - this time it is 95%
c = 0.95

n = 10

# We assume that daily returns are normally distributed
mu = np.mean(stock_data['Returns'])
sigma = np.std(stock_data['Returns'])

# Calculate and print the Value at Risk (VaR)
var = calculate_VaR_n(S, c, mu, sigma, n)
print(f'Value at Risk is: ${var.round(2)}')

Value at Risk is: $82897.97


## VaR with Monte-Carlo Simulations

In [8]:
import numpy as np
import yfinance as yf
import pandas as pd
import datetime

In [9]:
def download_data(stock, start_date, end_date):
    data = {}

    # Download stock data using yfinance
    ticker = yf.download(stock, start_date, end_date)

    # Extract the 'Adj Close' prices and store them in the data dictionary
    data[stock] = ticker['Adj Close']

    # Create a DataFrame from the data dictionary
    return pd.DataFrame(data)

In [10]:
class ValueAtRiskMonteCarlo:
    def __init__(self, S, mu, sigma, c, n, iterations):
        # This is the value of the initial investment at t = 0 (1000 dollars)
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.c = c
        self.n = n
        self.iterations = iterations

    def simulation(self):
        # Generate random numbers from a standard normal distribution
        rand = np.random.normal(0, 1, [1, self.iterations])

        # Equation for the S(t) stock price
        # The random walk of our initial investment
        stock_price = self.S * np.exp(self.n * (self.mu - 0.5 * self.sigma ** 2) +
                                      self.sigma * np.sqrt(self.n) * rand)
        
        # Sort the stock prices to determine the percentile
        stock_price = np.sort(stock_price)
        
        # Calculate the VaR based on the specified confidence level
        percentile = np.percentile(stock_price, (1 - self.c) * 100)

        return self.S - percentile

In [11]:

# Define the download_data and ValueAtRiskMonteCarlo classes (as mentioned earlier)

S = 1e6
c = 0.99
n = 1
iterations = 100000

start_date = datetime.datetime(2014, 1, 1)
end_date = datetime.datetime(2017, 1, 1)

# Download stock data for the symbol 'C' from start_date to end_date
citi = download_data('C', start_date, end_date)

# Calculate the logarithmic returns using the 'Adj Close' prices
citi['Returns'] = np.log(citi['C'] / citi['C'].shift(1))

# Remove the first row with NaN values
citi = citi[1:]

# Calculate the mean and standard deviation of the returns
mu = np.mean(citi['Returns'])
sigma = np.std(citi['Returns'])

# Create an instance of the ValueAtRiskMonteCarlo class
model = ValueAtRiskMonteCarlo(S, mu, sigma, c, n, iterations)

# Perform the Monte Carlo simulation and print the result
var = model.simulation()
print(f'The Value at Risk using a Monte Carlo simulation for Citi is: ${var.round(2)}')

[*********************100%***********************]  1 of 1 completed
The Value at Risk using a Monte Carlo simulation for Citi is: $37083.03
